In [2]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [82]:
data_size = 100

training_dataset = TensorDataset(torch.randint(0, 10, (data_size * 10, 4)), torch.randint(0, 10, (data_size * 10,)))
test_dataset = TensorDataset(torch.randint(0, 10, (data_size, 4)), torch.randint(0, 10, (data_size,)))

print(f'[train_dataset] {training_dataset.tensors[0].shape, training_dataset.tensors[1].shape}')
print(f'[test_dataset] {test_dataset.tensors[0].shape, test_dataset.tensors[1].shape}')

batch_size = 16

training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

X, y = next(iter(training_dataloader))
print(X.shape, y.shape)
print(f'[training_dataloader] {training_dataloader.dataset[0]}')
print(f'[test_dataloader] {test_dataloader.dataset[0]}')

[train_dataset] (torch.Size([1000, 4]), torch.Size([1000]))
[test_dataset] (torch.Size([100, 4]), torch.Size([100]))
torch.Size([16, 4]) torch.Size([16])
[training_dataloader] (tensor([3, 2, 7, 3]), tensor(9))
[test_dataloader] (tensor([3, 2, 4, 7]), tensor(0))


In [83]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'[device] {device}')

class RandomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc = nn.Sequential(
      nn.Linear(4, 16),
      nn.ReLU(),
      nn.Linear(16, 10),
    )
    
  def forward(self, x):
    x = x / 10
    return self.fc(x)

model = RandomModel().to(device)
print(f'[model] {model}')

[device] cuda
[model] RandomModel(
  (fc): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=10, bias=True)
  )
)


In [84]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [85]:
def train(model: RandomModel, training_dataloader, loss_fn: nn.CrossEntropyLoss, optimizer):
  for batch_idx, (X, y) in enumerate(training_dataloader):
    optimizer.zero_grad()
    X, y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)

    loss.backward()
    optimizer.step()

    if batch_idx % 1 == 0:
      print(f'[{batch_idx + 1}/{len(training_dataloader)}] Loss: {loss.item():.4f}')

In [86]:
def test(model: RandomModel, test_dataloader, loss_fn: nn.CrossEntropyLoss):
  size = len(test_dataloader.dataset)
  for batch_idx, (X, y) in enumerate(test_dataloader):
    model.eval()
    test_loss, correct = 0., 0.
    with torch.no_grad():
      for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        y_pred: torch.Tensor = model(X)
        loss_value: float = loss_fn(y_pred, y).item()
        test_loss += loss_value
        print(f'test : {y_pred.argmax(1), y}')
        correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()
  print(f'[test] Loss: {test_loss:.4f} Acc: {correct/size:.4f}')

In [87]:
train(model, training_dataloader, loss_fn, optimizer)
test(model, test_dataloader, loss_fn)

[1/63] Loss: 2.3312
[2/63] Loss: 2.2944
[3/63] Loss: 2.2827
[4/63] Loss: 2.3623
[5/63] Loss: 2.3468
[6/63] Loss: 2.3621
[7/63] Loss: 2.2736
[8/63] Loss: 2.2637
[9/63] Loss: 2.3049
[10/63] Loss: 2.3317
[11/63] Loss: 2.3883
[12/63] Loss: 2.3300
[13/63] Loss: 2.3424
[14/63] Loss: 2.2609
[15/63] Loss: 2.3280
[16/63] Loss: 2.2304
[17/63] Loss: 2.2719
[18/63] Loss: 2.3268
[19/63] Loss: 2.3283
[20/63] Loss: 2.3373
[21/63] Loss: 2.3388
[22/63] Loss: 2.3188
[23/63] Loss: 2.3411
[24/63] Loss: 2.2956
[25/63] Loss: 2.3229
[26/63] Loss: 2.3546
[27/63] Loss: 2.3033
[28/63] Loss: 2.2890
[29/63] Loss: 2.3057
[30/63] Loss: 2.1913
[31/63] Loss: 2.2494
[32/63] Loss: 2.3399
[33/63] Loss: 2.3455
[34/63] Loss: 2.3157
[35/63] Loss: 2.2638
[36/63] Loss: 2.2749
[37/63] Loss: 2.3323
[38/63] Loss: 2.3210
[39/63] Loss: 2.3177
[40/63] Loss: 2.3178
[41/63] Loss: 2.3498
[42/63] Loss: 2.2859
[43/63] Loss: 2.2778
[44/63] Loss: 2.2882
[45/63] Loss: 2.2645
[46/63] Loss: 2.3282
[47/63] Loss: 2.2580
[48/63] Loss: 2.3491
[